In [1]:
import os,json
import spacy
from tqdm import tqdm
nlp = spacy.load("en_core_web_sm")

# CoNLL03

In [2]:
path_folder='/home/xhsun/NLP/NER/NerData/en_conll03'

In [5]:
def parsing_sentence(sentence_list):
    sentence=' '.join(sentence_list)
    doc=nlp(sentence)
    result=[]
    for token in doc:
        word=token.text
        deprel=token.dep_.lower()
        head=token.head.text
        pos_tag=token.pos_
        try:
            head_id=sentence_list.index(str(head))
            #容易出现parsing错误的情况
        except:
            return None
        if deprel=='root':
            head_id=0
        else:
            head_id+=1
        result.append([word,pos_tag,head_id,deprel])
    return result

def process_data(path_folder,write_path):
    files_path=os.listdir(path_folder)
    print(files_path)
    for file_name in files_path:
        with open(os.path.join(path_folder,file_name)) as f:
            lines=f.readlines()
        sentences_and_entlabels=[([],[])]
        for line in lines:
            if line.strip() in ['',' ']:
                sentences_and_entlabels.append(([],[]))
            else:
                line_split=line.strip().split()
                assert len(line_split)==2
                word,entity_label=line_split
                sentences_and_entlabels[-1][0].append(word)
                sentences_and_entlabels[-1][1].append(entity_label)
                
        if sentences_and_entlabels[-1]==([],[]):
            del sentences_and_entlabels[-1]
            
        with open(os.path.join(write_path,file_name+".conllx"),'w') as f:
            parsing_error_count=0
            for example in tqdm(sentences_and_entlabels):
                sentences,entlabels=example
                assert len(sentences)==len(entlabels)
                
                parsing_result=parsing_sentence(sentence_list=sentences)
                if parsing_result==None or len(parsing_result)!=len(entlabels):
                    #parsing出现错误
                    parsing_error_count+=1
                    continue
                    
                for i in range(len(parsing_result)):
                    word,pos_tag,head_id,deprel=parsing_result[i]
                    ent=entlabels[i]
                    lemma='_'
                    feats='_'
                    conllx_example=[str(i+1),word,lemma,pos_tag,pos_tag,feats,str(head_id),deprel,'_','_',ent]
                    f.write('\t'.join(conllx_example)+'\n')
                    
                f.write('\n')
            print("parsing error count : ",parsing_error_count,len(sentences_and_entlabels))

In [6]:
process_data(path_folder=path_folder,write_path='/home/xhsun/Desktop/gitRepositories/DP-Relatedwork-BrieflySummarize/data/CoNLL03/')

['train.word.bmes', 'test.word.bmes', 'dev.word.bmes']


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 14041/14041 [00:53<00:00, 260.64it/s]


parsing error count :  4026 14041


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 3453/3453 [00:13<00:00, 256.86it/s]


parsing error count :  979 3453


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 3250/3250 [00:13<00:00, 249.72it/s]

parsing error count :  1030 3250


In [32]:
doc=nlp("Brussels 1996-08-22")

In [36]:
for token in doc:
    print(token,token.head,token.dep_)

Brussels Brussels ROOT
1996 Brussels nummod
- Brussels punct
08 22 nummod
- 22 punct
22 Brussels appos
